In [1]:
import pandas as pd 
import numpy as np



In [2]:
faults = pd.read_csv('./data/J1939Faults.csv')
onboard = pd.read_csv('./data/VehicleDiagnosticOnboardData.csv')

C:\Users\jacks\AppData\Local\Temp\ipykernel_7576\3888359493.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv('./data/J1939Faults.csv')


In [3]:
onboard.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [4]:
faults.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
dtype: object

In [5]:
on_faults = pd.merge(faults, onboard, left_on = 'RecordID', right_on = 'FaultId').drop(columns = ['Id', 'FaultId'])

In [7]:


# Get rid of the rows where the EquipmentID is more than 5 characters
on_faults_filtered = on_faults.loc[on_faults['EquipmentID'].astype(str).str.len() <= 5]

# Filter out faults where the light is going off
on_faults_filtered = on_faults_filtered.loc[on_faults['active'] == True].reset_index(drop = True)



In [8]:
on_faults_filtered['EquipmentID'].nunique()

1841

In [9]:
on_faults_filtered.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
Name                      object
Value                     object
dtype: object

In [10]:
# Convert EventTimeStamp column to datetime object
on_faults_filtered['EventTimeStamp'] = pd.to_datetime(on_faults_filtered['EventTimeStamp'])


In [11]:
# Set EquipmentID and EventTimeStamp columns as index
on_faults_filtered.set_index(['EquipmentID', 'EventTimeStamp', 'spn'], inplace=True)

In [12]:

# Pivot the data into a time series format
full_derates_ts = on_faults_filtered.pivot_table(index=['EquipmentID', 'EventTimeStamp', 'spn'],
                                          values=['RecordID', 'ESS_Id', 'eventDescription', 
                                                 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 
                                                 'ecuMake', 'ecuSource', 'fmi', 'active', 'activeTransitionCount', 
                                                 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp'])

# Reset the index
full_derates_ts.reset_index(inplace=True)

# Sort the data by EquipmentID and EventTimeStamp
full_derates_ts.sort_values(by=['EquipmentID', 'EventTimeStamp'], inplace=True)

# Group the data by EquipmentID
faults_grouped = full_derates_ts.groupby('EquipmentID')

# Create a dictionary of time series, with EquipmentID as the key and the time series as the value
faults_ts_dict = {group: data.drop('EquipmentID', axis=1).set_index('EventTimeStamp').sort_index() 
               for group, data in faults_grouped}


# Fill missing values with 0
for ts in faults_ts_dict.values():
    ts.fillna(0, inplace=True)



C:\Users\jacks\AppData\Local\Temp\ipykernel_7576\856689262.py:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  full_derates_ts = on_faults_filtered.pivot_table(index=['EquipmentID', 'EventTimeStamp', 'spn'],


In [15]:
faults_ts_dict.get(301) #This info would be for one specific truck ID

,spn,ESS_Id,Latitude,Longitude,MCTNumber,RecordID,active,activeTransitionCount,ecuSource,fmi
EventTimeStamp,,,,,,,,,,
2015-04-26 06:16:16,190,2092800.0,36.066527,-86.435277,105431712,34467.0,1.0,126.0,49.0,0.0
2015-04-28 05:29:21,190,2123154.0,36.066666,-86.435000,105431712,36192.0,1.0,126.0,49.0,0.0
2015-05-10 07:11:34,639,2346346.0,36.067083,-86.433842,105431712,48298.0,1.0,127.0,11.0,2.0
2015-05-11 13:11:20,639,2363162.0,36.189398,-82.795601,105431712,49415.0,1.0,127.0,11.0,2.0
2015-05-13 08:22:32,596,2400445.0,35.872500,-84.475648,105431712,51363.0,1.0,3.0,49.0,31.0
...,...,...,...,...,...,...,...,...,...,...
2020-02-16 08:58:40,829,121207314.0,36.066851,-86.434907,105441522,1242486.0,1.0,126.0,49.0,3.0
2020-02-18 11:02:29,639,121499858.0,35.587870,-86.443194,105441522,1243910.0,1.0,1.0,0.0,9.0
2020-02-18 11:13:23,1231,121502192.0,35.587870,-86.443194,105441522,1243916.0,1.0,14.0,49.0,16.0


Now we have a dictionary where the keys are the specific Equipment Id's and the info inside of them pertians to the types of service codes that came up with a timestamp value. That way we can se if there is info preceding a 5246 "breakdown" code that could predict it coming ahead of time. 

Lets look at one of the trucks with the most data inculuded in 